# Preprocess original census data 2017
- Open original census data
- Extract all rows for maize
- Rename variables to english
- Add variables of interest (yield, country_code, harvest_year)
- Save file as csv

In [1]:
# Imports
import pandas as pd
from pathlib import Path
import numpy as np

In [2]:
# Paths
original_path = Path.cwd().parent / 'original_data' 
original_path

PosixPath('/home/vant/Documents/valencia/agml_workshop/inegi_censos/original_data')

In [3]:
# Replace 'file_path.xlsx' with the path to your Excel file
file_path = original_path/'ena17_ent_agri03.xlsx'

# Read the Excel file into a Pandas DataFrame
df = pd.read_excel(file_path,skiprows=5)

In [4]:
df.head()

,Entidad,Cultivo,Entidad federativa y cultivo,Superficie cultivada,Unnamed: 4,Producción,Disponibilidad del agua,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,NaN,NaN,NaN,NaN,NaN,NaN,Temporal,NaN,NaN,Riego,NaN,NaN
1,NaN,NaN,NaN,Superficie sembrada,Superficie cosechada,NaN,Superficie sembrada,Superficie cosechada,Producción,Superficie sembrada,Superficie cosechada,Producción
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Hectáreas,NaN,Toneladas,Hectáreas,NaN,Toneladas,Hectáreas,NaN,Toneladas
4,NaN,NaN,NaN,A = D + G,B = E + H,C = F + I,D,E,F,G,H,I


In [5]:
df.columns

Index(['Entidad', 'Cultivo', 'Entidad federativa y cultivo',
       'Superficie cultivada', 'Unnamed: 4', 'Producción',
       'Disponibilidad del agua', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11'],
      dtype='object')

In [6]:
# Define new column names
column_names = ['Entidad federativa', 'Cultivo','Entidad federativa y cultivo','Total superficie sembrada','Total superficie cosechada',
                'Producción total','MH-temporal superficie sembrada','MH-temporal superficie cosechada','MH-temporal producción',
                'MH-riego superficie sembrada','MH-riego superficie cosechada','MH-riego producción']

# Rename the columns
df.columns = column_names

# Delete rows with initial no data
df = df.drop(index=range(5)).reset_index(drop=True)

In [7]:
df.head()

,Entidad federativa,Cultivo,Entidad federativa y cultivo,Total superficie sembrada,Total superficie cosechada,Producción total,MH-temporal superficie sembrada,MH-temporal superficie cosechada,MH-temporal producción,MH-riego superficie sembrada,MH-riego superficie cosechada,MH-riego producción
0,01 Ags,NaN,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01 Ags,NaN,Anuales,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01 Ags,Frijol,Frijol,3358.74626,3201.45101,2290.601174,2409.01026,2251.71501,1120.357124,949.736,949.736,1170.24405
3,01 Ags,Maíz blanco,Maíz blanco,73393.422331,67484.245731,201502.371934,61353.990581,55444.813981,82382.615314,12039.43175,12039.43175,119119.75662
4,01 Ags,NaN,Perennes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Extract key:value for state code

# Filter rows when Cultivo es NaN
df.dropna(subset=['Entidad federativa', 'Entidad federativa y cultivo'], inplace=True)
estado_codigo_nombre = df.loc[df['Cultivo'].isna(), ['Entidad federativa', 'Entidad federativa y cultivo']]
estado_codigo_nombre = estado_codigo_nombre[~estado_codigo_nombre['Entidad federativa y cultivo'].isin(["Perennes", "Anuales"])]
estado_codigo_nombre.columns = ["state_code","state_name"] 

# Split 'codigo' into 'adm_id' and 'state_acrs'
estado_codigo_nombre[['adm_id', 'state_acrs']] = estado_codigo_nombre['state_code'].str.split(' ', expand=True)

estado_codigo_nombre['adm_id'] = estado_codigo_nombre['adm_id'].astype(str)

# add campeche state which is missing

missing_state_data = {
    'state_code': '04 Camp',
    'state_name': 'Campeche',
    'adm_id': '04',
    'state_acrs': 'Camp'
}

missing_state_df = pd.DataFrame([missing_state_data])
estado_codigo_nombre = pd.concat([estado_codigo_nombre,missing_state_df],ignore_index=True)

# show dataframe
print(estado_codigo_nombre)

# Save key df
estado_codigo_nombre.to_csv('state_key.csv', index=False)

   state_code                       state_name adm_id state_acrs
0      01 Ags                   Aguascalientes     01        Ags
1       02 BC                  Baja California     02         BC
2      03 BCS              Baja California Sur     03        BCS
3      05 Coa             Coahuila de Zaragoza     05        Coa
4      06 Col                           Colima     06        Col
5      07 Chs                          Chiapas     07        Chs
6      08 Chi                        Chihuahua     08        Chi
7      09 CMX                 Ciudad de México     09        CMX
8      10 Dgo                          Durango     10        Dgo
9      11 Gto                       Guanajuato     11        Gto
10     12 Gro                         Guerrero     12        Gro
11     13 Hgo                          Hidalgo     13        Hgo
12     14 Jal                          Jalisco     14        Jal
13     15 Mex                           México     15        Mex
14     16 Mic            

In [9]:
df.dropna(subset=['Cultivo'], inplace=True)
# Drop the column at index 3
df_clean = df.drop("Entidad federativa y cultivo", axis=1)
df_clean.head()


,Entidad federativa,Cultivo,Total superficie sembrada,Total superficie cosechada,Producción total,MH-temporal superficie sembrada,MH-temporal superficie cosechada,MH-temporal producción,MH-riego superficie sembrada,MH-riego superficie cosechada,MH-riego producción
2,01 Ags,Frijol,3358.74626,3201.45101,2290.601174,2409.01026,2251.71501,1120.357124,949.736,949.736,1170.24405
3,01 Ags,Maíz blanco,73393.422331,67484.245731,201502.371934,61353.990581,55444.813981,82382.615314,12039.43175,12039.43175,119119.75662
5,01 Ags,Guayaba,3887.692187,3520.011687,33612.830537,0,0,0,3887.692187,3520.011687,33612.830537
8,02 BC,Cebolla,2873.0976,2850.1976,95135.28156,0,0,0,2873.0976,2850.1976,95135.28156
11,03 BCS,Jitomate,1195.71095,1172.96095,82809.648064,0,0,0,1195.71095,1172.96095,82809.648064


In [10]:
# Extract only maize
maiz_df = df_clean[df_clean['Cultivo'].str.contains('Maíz', case=False)].copy()
maiz_df.head(5)

,Entidad federativa,Cultivo,Total superficie sembrada,Total superficie cosechada,Producción total,MH-temporal superficie sembrada,MH-temporal superficie cosechada,MH-temporal producción,MH-riego superficie sembrada,MH-riego superficie cosechada,MH-riego producción
3,01 Ags,Maíz blanco,73393.422331,67484.245731,201502.371934,61353.990581,55444.813981,82382.615314,12039.43175,12039.43175,119119.75662
12,03 BCS,Maíz blanco,4644.99695,4582.99695,33856.84485,182.8334,173.8334,93.83345,4462.16355,4409.16355,33763.0114
18,05 Coa,Maíz blanco,68820.574663,37634.2971,NaN,52657.822338,24258.235475,NaN,16162.752325,13376.061625,NaN
24,06 Col,Maíz blanco,16200.289808,16034.177048,54795.656418,13304.88757,13138.77481,43156.571618,2895.402238,2895.402238,11639.0848
30,07 Chs,Maíz blanco,519026.048064,512396.603064,1622396.828905,494760.214889,488161.129639,1488474.65536,24265.833175,24235.473425,133922.173544


In [11]:
maiz_df.shape

(27, 11)

In [12]:
# replace Entidad federativa codes for state names
maiz_df.loc[:,'Entidad federativa'] = maiz_df['Entidad federativa'].map(estado_codigo_nombre.set_index('state_code')['adm_id'])
maiz_df.head()

,Entidad federativa,Cultivo,Total superficie sembrada,Total superficie cosechada,Producción total,MH-temporal superficie sembrada,MH-temporal superficie cosechada,MH-temporal producción,MH-riego superficie sembrada,MH-riego superficie cosechada,MH-riego producción
3,01,Maíz blanco,73393.422331,67484.245731,201502.371934,61353.990581,55444.813981,82382.615314,12039.43175,12039.43175,119119.75662
12,03,Maíz blanco,4644.99695,4582.99695,33856.84485,182.8334,173.8334,93.83345,4462.16355,4409.16355,33763.0114
18,05,Maíz blanco,68820.574663,37634.2971,NaN,52657.822338,24258.235475,NaN,16162.752325,13376.061625,NaN
24,06,Maíz blanco,16200.289808,16034.177048,54795.656418,13304.88757,13138.77481,43156.571618,2895.402238,2895.402238,11639.0848
30,07,Maíz blanco,519026.048064,512396.603064,1622396.828905,494760.214889,488161.129639,1488474.65536,24265.833175,24235.473425,133922.173544


In [13]:
# translate colnames to english
english_col_names = ['adm_id',
                     'crop_name',
                     'planted_area',
                     'harvest_area',
                     'production',
                     'Water Modality - Temporary - Cultivated area - Sown',
                     'Water Modality - Temporary - Cultivated area - Harvested',
                     'Water Modality - Temporary - Production',
                     'Water Modality - Irrigation - Cultivated area - Sown',
                     'Water Modality - Irrigation - Cultivated area - Harvested',
                     'Water Modality - Irrigation - Production']

maiz_df.columns = english_col_names

# translate to English crop names
# Define translations
translations = {
    'Maíz forrajero': 'Forage corn',
    'Maíz amarillo': 'Yellow corn',
    'Maíz blanco': 'White corn'
}

# Replace the values in the "Cultivo" column with their English translations
maiz_df.loc[:, "crop_name"] = maiz_df["crop_name"].replace(translations)

maiz_df.head(5)

,adm_id,crop_name,planted_area,harvest_area,production,Water Modality - Temporary - Cultivated area - Sown,Water Modality - Temporary - Cultivated area - Harvested,Water Modality - Temporary - Production,Water Modality - Irrigation - Cultivated area - Sown,Water Modality - Irrigation - Cultivated area - Harvested,Water Modality - Irrigation - Production
3,01,White corn,73393.422331,67484.245731,201502.371934,61353.990581,55444.813981,82382.615314,12039.43175,12039.43175,119119.75662
12,03,White corn,4644.99695,4582.99695,33856.84485,182.8334,173.8334,93.83345,4462.16355,4409.16355,33763.0114
18,05,White corn,68820.574663,37634.2971,NaN,52657.822338,24258.235475,NaN,16162.752325,13376.061625,NaN
24,06,White corn,16200.289808,16034.177048,54795.656418,13304.88757,13138.77481,43156.571618,2895.402238,2895.402238,11639.0848
30,07,White corn,519026.048064,512396.603064,1622396.828905,494760.214889,488161.129639,1488474.65536,24265.833175,24235.473425,133922.173544


In [14]:
# Create country and year column
maiz_df["country_code"] = "MX"
maiz_df["harvest_year"] = "2017"

# calculate yield
maiz_df["yield"] = maiz_df["production"] / maiz_df["harvest_area"]

# reorder columns and leave the target ones
ordered_columns = [
    'crop_name','country_code','adm_id', 
    'planted_area', 'harvest_area','harvest_year', 'yield','production'
]

# Reorder the DataFrame columns
maiz_df = maiz_df[ordered_columns]



In [15]:
# Transform to numeric 
maiz_df['planted_area'] = pd.to_numeric(maiz_df['planted_area'], errors='coerce')
maiz_df['harvest_area'] = pd.to_numeric(maiz_df['harvest_area'], errors='coerce')
maiz_df['yield'] = pd.to_numeric(maiz_df['yield'], errors='coerce')
maiz_df['production'] = pd.to_numeric(maiz_df['production'], errors='coerce')
maiz_df['production'] = maiz_df['production']/1000000
# Add prefix "MX-" to the 'adm_id' column# Convert 'adm_id' column to string type
maiz_df['adm_id'] = maiz_df['adm_id'].astype(str)
maiz_df['adm_id'] = maiz_df['adm_id'].apply(lambda x: 'MX' + x)
maiz_df.head()

,crop_name,country_code,adm_id,planted_area,harvest_area,harvest_year,yield,production
3,White corn,MX,MX01,73393.422331,67484.245731,2017,2.985917,0.201502
12,White corn,MX,MX03,4644.996950,4582.996950,2017,7.387490,0.033857
18,White corn,MX,MX05,68820.574663,37634.297100,2017,NaN,NaN
24,White corn,MX,MX06,16200.289808,16034.177048,2017,3.417429,0.054796
30,White corn,MX,MX07,519026.048064,512396.603064,2017,3.166291,1.622397


In [16]:
# check data for nan
nan_counts = maiz_df.isna().sum()
print(nan_counts)

crop_name       0
country_code    0
adm_id          0
planted_area    0
harvest_area    0
harvest_year    0
yield           2
production      2
dtype: int64


In [17]:
maiz_df.shape

(27, 8)

In [18]:
# Define metadata
metadata = {
    "source":"INEGI Encuesta Nacional Agropecuaria 2017",
    "yield": "tonnes/ha",
    "production": "megatonnes",
    "Areas": "hectares",
    "Note": "Last update was on 8th of January,2019 since there was en error found and fixed"
}

# Store metadata in attributes or dictionaries
maiz_df.attrs['metadata'] = metadata

# Display the modified DataFrame
maiz_df.attrs

{'metadata': {'source': 'INEGI Encuesta Nacional Agropecuaria 2017',
  'yield': 'tonnes/ha',
  'production': 'megatonnes',
  'Areas': 'hectares',
  'Note': 'Last update was on 8th of January,2019 since there was en error found and fixed'}}

In [19]:
# Saving data
# Save DataFrame to CSV
maiz_df.to_csv('maize_data_2017.csv', index=False)

# Save metadata to a separate file (e.g., JSON)
import json
with open('maize_metadata_2017.json', 'w') as file:
    json.dump(metadata, file)

In [20]:
#Check saved data
# Load DataFrame from CSV
maiz_df2 = pd.read_csv('maize_data_2017.csv', index_col=0)

# Load metadata from JSON
with open('maize_metadata_2017.json', 'r') as file:
    metadata = json.load(file)

# Assign metadata back to the DataFrame
maiz_df2.attrs['metadata'] = metadata

maiz_df2.attrs
#maiz_df2.head()

{'metadata': {'source': 'INEGI Encuesta Nacional Agropecuaria 2017',
  'yield': 'tonnes/ha',
  'production': 'megatonnes',
  'Areas': 'hectares',
  'Note': 'Last update was on 8th of January,2019 since there was en error found and fixed'}}